<a href="https://colab.research.google.com/github/Haxr213/Universidade_Catolica_Santos/blob/master/6_Semestre/Inteligencia_Artificial/ACO_carrinho_compras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Importações**

In [1]:
import numpy as np
import pandas as pd
import random as rd
import math
import matplotlib.pyplot as plt

#Instalando o ACO-PANTS, deve ser executado pelo menos uma vez
!pip install ACO-PANTS
!pip install ipython-autotime
%load_ext autotime

import pants

  Created wheel for ACO-PANTS: filename=ACO_Pants-0.5.2-cp36-none-any.whl size=18854 sha256=b31ef99e11165fc71741198cd5d9663d5c987ca69e17ee198e7a291f52399508
  Stored in directory: /root/.cache/pip/wheels/96/54/43/24aef81d408a74657e0cc72e811dd69b4a8f1d20c3ae758129
Successfully built ACO-PANTS
time: 3.43 ms


### **Leitura da base de dados (arquivo CSV)**

In [2]:
url = "https://raw.githubusercontent.com/luisdurante/IA-II-algoritmo-genetico-carrinho-de-compras/master/amostra.csv"
df = pd.read_csv("https://raw.githubusercontent.com/luisdurante/IA-II-algoritmo-genetico-carrinho-de-compras/master/amostra.csv", sep=';')
df.head()

,produto,valor,prioridade,categoria,valor nutritivo
0,água,"R$ 1,99",5,bebidas,5
1,cerveja,"R$ 3,99",2,bebidas,1
2,refrigerante 2L,"R$ 6,00",5,bebidas,1
3,vodka,"R$ 30,00",1,bebidas,1
4,vinho,"R$ 60,00",2,bebidas,1


time: 330 ms


In [3]:
df.valor = df.valor.apply(lambda x: float(x.replace(',','.')[2:]))
df['valor nutritivo'] = df['valor nutritivo'].apply(lambda x: float(x))
df.valor.sum()
df.head()


,produto,valor,prioridade,categoria,valor nutritivo
0,água,1.99,5,bebidas,5.0
1,cerveja,3.99,2,bebidas,1.0
2,refrigerante 2L,6.00,5,bebidas,1.0
3,vodka,30.00,1,bebidas,1.0
4,vinho,60.00,2,bebidas,1.0


time: 23.1 ms


### **Elaboração do Pontos**

In [4]:
df.drop(columns=['produto', 'prioridade', 'categoria'], axis='1', inplace=True)
df.head()




,valor,valor nutritivo
0,1.99,5.0
1,3.99,1.0
2,6.00,1.0
3,30.00,1.0
4,60.00,1.0


time: 17.1 ms


In [5]:

nodes = []
records = df.to_records(index=False)
nodes = list(records)
print(nodes)


[(1.99, 5.), (3.99, 1.), (6., 1.), (30., 1.), (60., 1.), (48., 4.), (25., 5.), (35., 3.), (35.48, 4.), (30.68, 3.), (15., 2.), (3.5, 5.), (18., 5.), (18.13, 3.), (6., 3.), (5., 4.), (5.98, 3.), (5.09, 4.), (6.09, 4.), (7., 5.), (8., 3.), (5.24, 3.), (3., 4.), (6.15, 5.), (8.45, 5.), (7.07, 5.), (6.11, 5.), (8.32, 5.), (5.37, 2.), (33., 1.), (7.18, 1.), (3.17, 3.), (2., 2.), (14., 2.), (17., 4.), (8.15, 4.), (10., 4.), (8.3, 3.), (6.5, 3.)]
time: 12.8 ms


### **Função de Transição**

In [6]:
def nota(a, b):
    #Baseado em valor nutritivo
    return math.sqrt(pow((a[1]*5)/a[0], 2) + pow((b[1]*5)/b[0], 2))
  
    
    

time: 2.5 ms


### **Algoritmo para Melhor Resultado**

In [7]:
#Orçamento do cliente
precoLimite = 100

#Criação do mundo para as formigas
world = pants.World(nodes, nota)

#Construção do solver
solver = pants.Solver()

#Carrinho com a melhor solução
carrinho = []

#Variáveis de avaliação da solução
fatorben = 0.0
preco = 0.0
valor = 0.0
valornut = 0
benMinimo = 1.5

#Variáveis delimitadoras
iter = 10
i = 0
j = 0
#Utilizado junto ao beneficio minimo, garante que a solução atenda o custo 
#benefício mínimo esperado
halt = 1

time: 35.6 ms


### **Melhor Carrinho**

In [8]:
while (j < iter):
  #Faz a otimização
  solution = solver.solve(world)
  lista = (solution.tour)
  #Cria um carrinho válido (dentro do orçamento) particionando o vetor
  for x in lista:
    if ((valor + x[0]) <= precoLimite):
      valor = valor + x[0]
      valornut = valornut + x[1]
      i = i + 1
    else:
      #Avalia o carrinho, substituindo o antigo caso encontre um melhor
      if ((valornut*5/valor) > fatorben):
        carrinho.clear
        carrinho = (lista)[:i]
        fatorben = valornut*5/valor
        preco = valor
      if (fatorben >= benMinimo):
        halt = 0
      if (halt != 1):
        j = j + 1
      i = 0
      valor = 0.0
      valornut = 0
      break
      

print("De acordo com o orcamento de R$", precoLimite,"\b. Sugerimos este carrinho:", carrinho)
print("Com o preco de: R$", format(preco,".2f"))
print("Custo beneficio: ", fatorben)

De acordo com o orcamento de R$ 100 . Sugerimos este carrinho: [(1.99, 5.), (3.5, 5.), (2., 2.), (3.17, 3.), (5., 4.), (8.45, 5.), (30.68, 3.)]
Com o preco de: R$ 54.79
Custo beneficio:  2.463953276145282
time: 10.1 s
